# Non-negative Matrix Factorization: _Movie Recommendation_
_"What movie should I watch this evening?"_

---

The objective of this tutorial is to use in practice Non-Negative Matrix Factorization (NMF). More precisely, we will build a movie recommendation system, which is a standard problem where NMF is used.
Here, we apply NMF to user-item matrix, representing the ratings of different persons for different movies. The goal is to make good use of these data to predict the movies that a new user is likely to like.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib inline

import seaborn as sns
sns.set_theme(style="darkgrid")

___

## Data exploration

For this study, you will need to download the [MovieLens](https://en.wikipedia.org/wiki/MovieLens) dataset from the Grouplens site [grouplens.org/datasets/movielens](https://grouplens.org/datasets/movielens/). There are several datasets available on this page. In order to keep the digital cost under control, we will download [MovieLens 100k Dataset ](https://grouplens.org/datasets/movielens/100k/).

The MovieLens datasets were collected by the GroupLens Research Project at the University of [Minnesota Harper and Konstan (2015)](https://dl.acm.org/doi/10.1145/2827872). The data was collected through the MovieLens web site between September 19th, 1997 and April 22nd, 1998. Users who had less than 20 ratings or did not have complete demographic information were removed from this dataset. More information about this dataset can be found in the [ml-100k-README.txt](https://files.grouplens.org/datasets/movielens/ml-100k-README.txt) page on the Grouplens page.

### Load the dataset

##### <span style="color:purple">**Todo:** Download and unzip the `ml-100k.zip` file.</span>

This "small" dataset consists of 100 000 ratings. Each user has rated at least 20 movies. Simple demographic info for the users (age, gender, occupation) are also available.

##### <span style="color:purple">**Todo:** Read the `u.data` file using the `read_csv` method.</span>

We will only use the first three columns, which correspond respectively to `User_ID`, `Movie ID` and `Rating`.

In [ ]:
### TO BE COMPLETED ### 

names = ['User ID', 'Movie ID', 'Rating']
ratings_df = ...

In [ ]:
# %load solutions/load_ratings.py

##### <span style="color:purple">**Todo:** Check that the dataset contains 100k ratings.</span>

1. How many different users are in this dataset? 
2. How many different movies did they rate? 

In [ ]:
### TO BE COMPLETED ### 

n_ratings = ...
n_users = ...
n_items = ...

print("Total number of ratings in the dataset: %i" % (n_ratings))
print("Number of persons who rated movies: %i" % (n_users))
print("Number of rated movies: %i" % (n_items))

In [ ]:
# %load solutions/check_dataset.py

### Distribution of the ratings

##### <span style="color:purple">**Todo:** Study the distribution of ratings.</span>

1. On what scale are the ratings given?
2. What is the mean score? median?

You may use the `describe()` method.

In [ ]:
### TO BE COMPLETED ### 
# Statistics of the ratings

ratings_df["Rating"].describe()

[...]

In [ ]:
# %load solutions/stats_ratings.py

##### <span style="color:purple">**Todo:** Plot the histogram of the ratings.</span>

Should we normalize these values before applying NMF?

In [ ]:
### TO BE COMPLETED ### 
# Histogram of the ratings

[...]

In [ ]:
# %load solutions/hists_ratings.py

### Ratings per user

##### <span style="color:purple">**Todo:** Plot the histogram representing the number of ratings per user.</span>

In [ ]:
### TO BE COMPLETED ### 
# Histogram of the ratings per user

users_list = ...
n_ratings_ind = ... # Number of movies rated by the user i
n_ratings_ind = np.array(n_ratings_ind)

[...]

In [ ]:
# %load solutions/hists_ratings_user.py

##### <span style="color:purple">**Question:** How many movies did people rate on average?</span>

Other statistics: min, max, median, quartiles?

In [ ]:
### TO BE COMPLETED ### 
# Statistics of the ratings per user

[...]

In [ ]:
# %load solutions/stats_ratings_user.py

##### <span style="color:purple">**Question:** What is the profile of the user who has rated the most films?</span>

The `u.user` file contains the demographic information for each user. We load it with the command below.

In [ ]:
# Loading demographic information about the users

names = ['User ID', 'Age', 'Gender', 'Occupation']
users_df = pd.read_csv("ml-100k/u.user", sep="|", header=None, usecols=[0,1,2,3], 
                           encoding='iso-8859-1', names=names)

users_df.head()

In [ ]:
### TO BE COMPLETED ### 
# User who rated the most films

[...]

In [ ]:
# %load solutions/max_ratings.py

### Average ratings per movie

##### <span style="color:purple">**Todo:** Plot the histogram representing the average rating per movie.</span>

In [ ]:
### TO BE COMPLETED ### 
# Histogram of the ratings per user

movies_list = ...
mean_ratings = ... # Average rating of the k-th movie

[...]

In [ ]:
# %load solutions/hists_ratings_movie.py

##### <span style="color:purple">**Todo:** Find movies with ratings 1 and 5.</span>

The movie titles can be read from the `u.item` file, using the following line of code: 

In [ ]:
# Loading movies' genre
genre_df = pd.read_csv("ml-100k/u.genre", sep="|", header=None, usecols=[0], encoding='latin-1', names=["Genre"])
genre = genre_df.values.squeeze().tolist()

# Loading movies 
cols = [1] + np.arange(5,24).tolist()
names = ["Title"] + np.arange(19).tolist()
movies_df = pd.read_csv("ml-100k/u.item", sep="|", header=None, usecols=cols, encoding='iso-8859-1', names=names)

# Each movie is assigned its first genre encountered in the list, in alphabetical order (arbitrary choice).
movies_df.insert(1, "Genre", movies_df.eq(1).idxmax(axis=1).values.squeeze().tolist(), True)

# Rename dataframe columns
movies_df.columns = ["Title","Genre"]+genre

movies_df.head()
# movies_df['Title'].head()

In [ ]:
### TO BE COMPLETED ### 
# Example of a *bad* and a *good* movie

[...]

In [ ]:
# %load solutions/bad_good_movies.py

## Applying Non-negative Matrix Factorization (NMF)

Most recommendation models consist in building a user-by-item matrix with some sort of "interaction"
number in each cell. Here, users give items numerical ratings, this is called an explicit feedback model.

In [ ]:
from scipy import sparse

### User-item matrix

<a id='user-item-matrix'></a>

##### <span style="color:purple">**Todo:** Build the user-item matrix for the movie rating problem studied here.</span>

- We can use the [`scipy.sparse.csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) function to build a sparse matrix. This matrix can then be converted to dense using the `todense()` method.
- Note that there is no user '0' in the dataset.

<br>

The command `print(R[:10,:20])` returns
```
[[5 3 4 3 3 5 4 1 5 3 2 5 5 5 5 5 3 4 5 4]
 [4 0 0 0 0 0 0 0 0 2 0 0 4 4 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0]
 [4 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0]
 [4 0 0 0 0 0 2 4 4 0 0 4 2 5 3 0 0 0 4 0]
 [0 0 0 5 0 0 5 5 5 4 3 5 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0 3 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [4 0 0 4 0 0 4 0 4 0 4 5 3 0 0 4 0 0 0 0]]
```
What should be the size of this matrix ?

In [ ]:
### TO BE COMPLETED ### 
# R user-item matrix

[...]

In [ ]:
# %load solutions/R_matrix.py

##### <span style="color:purple">**Todo:** Choose a pair (user,movie) and check that the rating in the R-matrix matches the rating in the dataset.</span>

For example, we can look at the first row of the dataset.

In [ ]:
### TO BE COMPLETED ### 

[...]

In [ ]:
# %load solutions/R_matrix_check.py

##### <span style="color:purple">**Question:** What is the sparsity, _i.e._ the proportion of zeros, of the matrix obtained?</span>

- We can use the [`nonzero()`](https://numpy.org/doc/stable/reference/generated/numpy.nonzero.html) method.

What are your thoughts on this? Does it interfere with the chosen scoring system?

In [ ]:
### TO BE COMPLETED ### 

[...]

In [ ]:
# %load solutions/R_matrix_sparsity.py

### Implementation of NMF decomposition

In [ ]:
from sklearn.decomposition import NMF

##### <span style="color:purple">**Todo:** Apply NMF with 20 components.</span>

1. How many iterations to convergence?
2. Verify the shape of the feature matrix representing the different movies.
3. Verify the shape of the feature matrix representing the different users.

In [ ]:
### TO BE COMPLETED ### 
# NMF decomposition

[...]

In [ ]:
# %load solutions/nmf.py

### Topics of the different components 

The cell below displays the titles of the films most "strongly" associated with each of the 20 components of the decomposition. 

<a id='topic_plot'></a>

In [ ]:
from matplotlib import colors

For each of the "topics" found in the decomposition, we will represent the films that contribued most to it, colored according to their genre. To do this, we will use the color palette below.

In [ ]:
cmap = plt.get_cmap('tab20', len(genre))
plt.barh(genre, .5, color=cmap.colors, label=genre)

plt.title("Color by movie genre")
plt.show()

##### <span style="color:purple">**Question:** What do you think about the resulting decomposition?</span>

- Comment the above figure.
- What does the bar length represent?

In [ ]:
n_top_words = 5
fig, axes = plt.subplots(5, 4, figsize=(50, 60), sharex=True)
axes = axes.flatten()
cmap = plt.get_cmap('tab20', len(genre))

for i in range(H.shape[0]):
    topic = H[i, :]
    top_features = topic.argsort()[: -n_top_words - 1 : -1]
    top_features_names = [movies_df["Title"][j][:20] for j in top_features]
    top_features_scores = [topic[j] for j in top_features]
    top_features_genres = [movies_df["Genre"][j] for j in top_features]
    top_cmap = colors.ListedColormap(cmap.colors[top_features_genres])
    
    ax = axes[i]
    ax.barh(top_features_names, top_features_scores, height=0.7, color=top_cmap.colors)
    ax.set_title(f"Topic {i+1}", fontdict={"fontsize": 30})
    ax.invert_yaxis()
    ax.tick_params(axis="both", which="major", labelsize=20)
    for i in "top right left".split():
        ax.spines[i].set_visible(False)

plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
plt.show()

### Reconstructed user-item matrix.

##### <span style="color:purple">**Question:** What is the average reconstruction error?</span>

Consider only the true ratings, _i.e_ the ones present in the original dataset.

In [ ]:
### TO BE COMPLETED ### 
# Average reconstruction error

[...]

In [ ]:
# %load solutions/reconstruction_error.py

##### <span style="color:purple">**Todo:** Observe the range of values taken by `R_pred`.</span>

1. What do you notice?
2. Suggest an improvement.

In [ ]:
### TO BE COMPLETED ### 
# Range of values

[...]

In [ ]:
# %load solutions/reconstruction_error_range.py

In [ ]:
### TO BE COMPLETED ### 
# Improvement

[...]

In [ ]:
# %load solutions/improve_prediction.py

### Known User 

Let's start with the simplest situation: recommending new movies to a user already in our database.

##### <span style="color:purple">**Todo:** Select a user at random within the dataset and recommend five movies he has not seen.</span>

The recommended movies are the ones with highest reconstruction scores among the unseen movies. These predictions can be displayed as a dataframe: one column for the title, one for the prediction score and one for the genre of the predicted movie.

In [ ]:
### TO BE COMPLETED ### 
# Recommendations known user

n_recommendations = 5

user_idx = ...
user = users_df[users_df['User ID']==user_idx]

A = user.iloc[0]['Age']
G = user.iloc[0]['Gender']
O = user.iloc[0]['Occupation']
print("Recommended movies for user %i: A %i year old" % (user_idx, A), G, O,'\n')

# ----- #

[...]
    
predicted_movies

In [ ]:
# %load solutions/reco_known_user.py

##### <span style="color:purple">**Todo:** Display the predicted movies in a horizontal barplot.</span>

Using the [horizontal barplot of the topics](#topic_plot), draw a horizontal barplot where the length of the bars is given by the rating of the predicted movie, and its color by the genre of the movie.

In [ ]:
### TO BE COMPLETED ### 
# Horizontal barplot

[...]

In [ ]:
# %load solutions/horizontal_barplot.py

##### <span style="color:purple">**Todo:** Compare the prediction with his/her favorite movies.</span>

1. Build the dataframe of these 5 favorite movies
2. Draw for these favorite movies the analog of the previous figure
3. Comment

In [ ]:
### TO BE COMPLETED ### 
# Favorite movies

[...]

In [ ]:
# %load solutions/favorite_movies.py

In [ ]:
### TO BE COMPLETED ### 
# Visual comparison

[...]

In [ ]:
# %load solutions/visual_comparison.py

##### <span style="color:purple">**Todo:** Find the three "topics" used by the NMF algorithm to choose the recommendations.</span>

In [ ]:
### TO BE COMPLETED ### 
# Top topics

top_topics = ...

[...]

In [ ]:
# %load solutions/top_topics.py

As a reminder, these are the topics associated with the following films: 

In [ ]:
n_top_words = 5
fig, axes = plt.subplots(1, 3, figsize=(30, 10), sharex=True)
axes = axes.flatten()
cmap = plt.get_cmap('tab20', len(genre))

for idx, i in enumerate(top_topics):
    topic = H[i, :]
    top_features = topic.argsort()[: -n_top_words - 1 : -1]
    top_features_names = [movies_df["Title"][j][:20] for j in top_features]
    top_features_scores = [topic[j] for j in top_features]
    top_features_genres = [movies_df["Genre"][j] for j in top_features]
    top_cmap = colors.ListedColormap(cmap.colors[top_features_genres])
    
    ax = axes[idx]
    ax.barh(top_features_names, top_features_scores, height=0.7, color=top_cmap.colors)
    ax.set_title(f"Topic {i+1}", fontdict={"fontsize": 30})
    ax.invert_yaxis()
    ax.tick_params(axis="both", which="major", labelsize=20)
    for i in "top right left".split():
        ax.spines[i].set_visible(False)

plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
plt.show()

### New user with rating history

##### <span style="color:purple">**Todo:** Create a new user, or use the `new_user.txt` file.</span>

In [ ]:
### TO BE COMPLETED ### 
# New user

[...]

In [ ]:
# %load new_user.txt

##### <span style="color:purple">**Todo:** Make recommendations for this new user.</span>

1. Update the user-item matrix,
2. Retrain the NMF model with the new matrix,
3. Make 5 recommendation for this new user,
4. Visually compare predictions and favorite movies.

_Note: Beware of the values taken by the predicted scores!_

In [ ]:
### TO BE COMPLETED ### 
# Recompute NMF

[...]

In [ ]:
# %load solutions/nmf_new_user_history.py

In [ ]:
### TO BE COMPLETED ### 
# Make recommendations

[...]

In [ ]:
# %load solutions/reco_new_user_history.py

In [ ]:
### TO BE COMPLETED ### 
# Visual comparison

[...]

In [ ]:
# %load solutions/visual_comparison_new_user_history.py

In [ ]:
# Top topics

topics_used = newW[-1, :]
top_topics = topics_used.argsort()[: -3 - 1 : -1]
top_topics.sort()
print("Top topics: %i, %i, %i" % tuple(top_topics + 1))

### New user with no rating history

The last case is that of a new user with no rating history, but currently watching a movie from our corpus. 

Without a history of notes, we will not be able to proceed as before. The idea here is, by placing ourselves in the "movies embedding" induced by the NMF decomposition, to compute a dissimilarity measure between the different movies and to recommend a movie close to the currently watched movie, in the sense of this dissimilarity. The underlying intuition is that the movie embedding of the NMF decomposition induces a good understanding of the movie corpus. 

We use [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) to measure the similarity between movies. Other choices are possible.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

##### <span style="color:purple">**Todo:** Compute the similarity between movies.</span>

1. Compute the movie-to-movie similarity matrix using the [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function,
2. Display the image representing this similarity.

In [ ]:
### TO BE COMPLETED ### 
# Movies similarity

[...]

In [ ]:
# %load solutions/movies_similarity.py

##### <span style="color:purple">**Todo:** Recommend the most similar movie to what the new user is watching.</span>

The movie currently being watched can be chosen at random.

In [ ]:
### TO BE COMPLETED ### 
# Suggested movies

[...]

In [ ]:
# %load solutions/suggested_movies.py

In [ ]:
### TO BE COMPLETED ### 
# Visualise suggestion

[...]

In [ ]:
# %load solutions/visualise_suggestion.py

## Optimize NMF parameters <span style="color:purple">(To go further)</span>

The next step is to optimize different parameters influencing the NMF model. To do so, you can conduct a K-fold cross-validation. You may choose $K=5$.

More precisely, we aim to select the optimal parameters for the [NMF scikit function](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html): $\texttt{n_components}$, $\texttt{solver}$, $\texttt{init}$, $\texttt{beta_loss}$, $\texttt{alpha_W}$, $\texttt{alpha_H}$, and $\texttt{l1_ratio}$. You can choose three values per parameters and test all combinations through a grid search.

In [ ]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.metrics import mean_squared_error

The function below creates the [user-item matrix](#user-item-matrix) $R$ from the numpy-array matrices $X$ and $y$.

- $\texttt{X = ratings_df[['User ID','Movie ID']].values}$
- $\texttt{y = ratings_df['Rating'].values}$

In [ ]:
def make_R(X, y, shape):
    # X = ratings_df[['User ID','Movie ID']].values
    # y = ratings_df['Rating'].values
    
    matrix_sparse = sparse.csr_matrix((y, (X[:,0], X[:,1])), shape=(shape[0]+1,shape[1]+1))
    R = matrix_sparse.todense()
    R = np.array(R[1:, 1:])
    
    return R

##### <span style="color:purple">**Todo:** Split the data set into a training set and a test set.</span>

We will perform the cross-validation only on the training set. The test set will be used to validate a posteriori the procedure on data that were not used to train the NMF decomposition.

In [ ]:
### TO BE COMPLETED ### 
# Train-vs-Test

[...]

In [ ]:
# %load solutions/split_train_test.py

We will need to evaluate the error (RMSE) to adjust the hyperparameters.

In [ ]:
def get_rmse(pred, actual):
    pred = pred[actual.nonzero()].flatten()     # Ignore nonzero terms
    actual = actual[actual.nonzero()].flatten() # Ignore nonzero terms
    return np.sqrt(mean_squared_error(pred, actual))

##### <span style="color:purple">**Todo:** Find the optimal parameters.</span>

You can use the [`ShuffleSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) function.

In [ ]:
### TO BE COMPLETED ### 
# Cross-Validation

[...]

In [ ]:
# %load solutions/cross_validation.py

##### <span style="color:purple">**Todo:** Evaluate the model obtained on the test set.</span>


In [ ]:
### TO BE COMPLETED ### 
# Final evaluation

[...]

In [ ]:
# %load solutions/final_evaluation.py